## NEB

In [1]:
from ase.optimize import BFGS
from ccs_fit.ase_calculator.ccs_ase_calculator import CCS
from tqdm import tqdm
from ase import io
import json

struct = io.read('DFT.db@11')
with open("CCS_params.json", 'r') as f:
    CCS_params = json.load(f)

CCS_calc = CCS(CCS_params=CCS_params)

#struct.rattle(0.05)
struct.calc = CCS_calc

opt = BFGS(struct, trajectory='opt_init.traj')
opt.run(fmax=0.005)

      Step     Time          Energy         fmax
BFGS:    0 10:40:42      -85.459618        0.6044
BFGS:    1 10:40:42      -85.503678        0.5775


C:\Users\Fremo211\Anaconda3\envs\Tantalizing\lib\site-packages\ase\utils\__init__.py:62: FutureWarning: Please use atoms.cell.rank instead
  warnings.warn(warning)


BFGS:    2 10:40:42      -86.026060        0.3347
BFGS:    3 10:40:42      -86.063429        0.3567
BFGS:    4 10:40:43      -86.081261        0.4180
BFGS:    5 10:40:43      -86.105497        0.4613
BFGS:    6 10:40:43      -86.185281        0.5130
BFGS:    7 10:40:43      -86.273551        0.5194
BFGS:    8 10:40:43      -86.378695        0.4936
BFGS:    9 10:40:44      -86.435663        0.6597
BFGS:   10 10:40:44      -86.483288        0.7488
BFGS:   11 10:40:44      -86.560144        0.7867
BFGS:   12 10:40:45      -86.663152        0.6835
BFGS:   13 10:40:45      -86.835912        0.4456
BFGS:   14 10:40:45      -86.885695        0.4349
BFGS:   15 10:40:45      -86.902645        0.4148
BFGS:   16 10:40:46      -86.944337        0.3305
BFGS:   17 10:40:46      -86.989331        0.3081
BFGS:   18 10:40:46      -87.033864        0.2658
BFGS:   19 10:40:46      -87.057172        0.2774
BFGS:   20 10:40:47      -87.067331        0.2427
BFGS:   21 10:40:47      -87.077464        0.2160


True

In [4]:
print(struct)

Atoms(symbols='Li2Ni4O8', pbc=True, cell=[[4.89787582, 0.0, 0.0], [-0.86307966, 5.01492177, 0.0], [-0.00032756, -1.57953315, 5.37866883]], calculator=CCS(...))


In [3]:
import ase

!ase gui opt_init.traj

In [4]:
from ase.visualize import view
init = io.read("opt_init.traj")

view(init)

<Popen: returncode: None args: ['C:\\Users\\Fremo211\\Anaconda3\\envs\\Tanta...>

In [7]:
final = init.copy()

final_pos = final.get_positions()
final_pos[24,:] = [-0.1, 3.2, 11.9]

final.set_positions(final_pos)

# view(final)

final.calc = CCS_calc
opt = BFGS(final, trajectory='final_opt.traj')
opt.run(fmax=0.005)

      Step     Time          Energy         fmax
BFGS:    0 13:52:54     -561.357678        0.0317
BFGS:    1 13:52:55     -561.357798        0.0315
BFGS:    2 13:52:56     -561.368203        0.0199
BFGS:    3 13:52:58     -561.368764        0.0282
BFGS:    4 13:52:59     -561.368854        0.0302
BFGS:    5 13:53:01     -561.369026        0.0324
BFGS:    6 13:53:02     -561.369442        0.0349
BFGS:    7 13:53:03     -561.370316        0.0357
BFGS:    8 13:53:05     -561.371728        0.0315
BFGS:    9 13:53:06     -561.372986        0.0209
BFGS:   10 13:53:07     -561.373508        0.0138
BFGS:   11 13:53:09     -561.373654        0.0149
BFGS:   12 13:53:10     -561.373698        0.0146
BFGS:   13 13:53:11     -561.373834        0.0132
BFGS:   14 13:53:13     -561.374085        0.0111
BFGS:   15 13:53:14     -561.374688        0.0101
BFGS:   16 13:53:16     -561.375679        0.0128
BFGS:   17 13:53:17     -561.376472        0.0235
BFGS:   18 13:53:18     -561.376700        0.0223
B

True

In [1]:
!ase gui final_opt.traj

'ase' is not recognized as an internal or external command,
operable program or batch file.


In [15]:
from ase import io
from ase.neb import NEB
from ase.optimize import MDMin

# Read initial and final states:
initial = io.read('opt_init.traj')
final = io.read('final_opt.traj')

# Make a band consisting of 5 images:
images = [init]
images += [init.copy() for i in range(3)]
images += [final]
neb = NEB(images)

# Interpolate linearly the potisions of the three middle images:
neb.interpolate()

# Set calculators:
for image in images[1:4]:
    CCS_calc = CCS(CCS_params=CCS_params)
    image.calc = CCS_calc
    
# Optimize:
optimizer = MDMin(neb, trajectory='NEB.traj')
optimizer.run(fmax=0.05)

       Step     Time          Energy         fmax
MDMin:    0 16:24:00     -561.338510        0.1147
MDMin:    1 16:24:07     -561.339001        0.1116
MDMin:    2 16:24:13     -561.340382        0.1027
MDMin:    3 16:24:19     -561.342409        0.0893
MDMin:    4 16:24:25     -561.344762        0.0731
MDMin:    5 16:24:31     -561.347149        0.0568
MDMin:    6 16:24:37     -561.349372        0.0482


True

In [18]:
saddle_point = io.read('NEB.traj@-3')

view(saddle_point)

io.write("saddle.POSCAR", saddle_point)

In [17]:
neb_images = io.read('NEB.traj@-5:-1')
print(len(neb_images))

4


In [23]:
for i in range(3):
    view(neb_images[i+1])
    io.write(str(i+1) + ".POSCAR", neb_images[i+1])